In [1]:
import json
import numpy as np
from socket import socket
from a2gmeasurements import HelperA2GMeasurements, GimbalRS2, GpsSignaling
import time, timeit

In [2]:
meas_number = '1_AGAIN_2_40'

OPERATE GROUND GIMBAL WITH CMDS

In [8]:
myGimbal = GimbalRS2()
myGimbal.start_thread_gimbal()

Start receiving messages


In [9]:
myGimbal.setPosControl(yaw=1650, roll=0, pitch=131)

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1



True

In [10]:
myGimbal.stop_thread_gimbal()
time.sleep(0.05)
myGimbal.actual_bus.shutdown()

Stopped receiving messages


MEASUREMENTS

In [3]:
host = ''
myHelper = HelperA2GMeasurements('GROUND', host, DBG_LVL_0=False, 
                                 DBG_LVL_1=True, 
                                 IsGPS=True, 
                                 IsGimbal=True, 
                                 IsSignalGenerator=True,
                                 F0=28.95,
                                 L0=20)

uptime library not available, timestamps are relative to boot time and not to Epoch UTC
Traceback (most recent call last):
  File "c:\Users\jvjulian\MyInstalledPrograms\envs\gimbalRS2\lib\site-packages\can\interfaces\pcan\pcan.py", line 70, in <module>
    import uptime
ModuleNotFoundError: No module named 'uptime'


Start receiving messagesSetting Gimbal RS2...


Setting GPS...



PEAK SIGNAL GENERATOR CALIBRATION

In [4]:
# Turn On RF output
myHelper.inst.write('RF1\n')
time.sleep(2)
# Turn Off RF output
myHelper.inst.write('RF0\n')

to_save_now = json.dumps(myHelper.mySeptentrioGPS.NMEA_buffer[-1])
filename = 'MEASUREMENT_' + meas_number + '_SIGNAL_GENERATOR_CALIBRATION_PEAK'
f = open(filename + '.json', 'w')
f.write(to_save_now)
f.close()

ROTATION GROUND GIMBAL MEASUREMENT

In [4]:
# 9. Request Gimbal orientation
myHelper.myGimbal.request_current_position()
time.sleep(0.0015)

# Gimbal assumed to be not moving
yaw_now = myHelper.myGimbal.yaw
#roll_now = myHelper.myGimbal.roll

# 10. Move GROUND gimbal in 30 deg steps
N_ang_steps = 360/30
reset_yaw_gimbal = myHelper.send_N_azimuth_angles(int(yaw_now*10), 
                                                  131, 
                                                  int(N_ang_steps), 
                                                  meas_number=meas_number)

# 11. Make the gimbal move in the opposite direction to unfold cables

print('ANGLES TO UNFOLD CABLE IN GIMBAL: ', reset_yaw_gimbal)
for i in reset_yaw_gimbal:
    myHelper.myGimbal.setPosControl(yaw=i, pitch=131, roll=0)
    time.sleep(1)

myHelper.HelperA2GStopCom()
#input('Stop test?')

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Pitch: 13.100000000000001, Yaw: 165.8, Roll: 0.0

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Measuring for 10  secs

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Pitch: 13.100000000000001, Yaw: -164.3, Roll: 0.0

Message sent on PCAN_USBBUS1

Measuring for 10  secs

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Pitch: 13.100000000000001, Yaw: -134.20000000000002, Roll: 0.0

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Message sent on PCAN_USBBUS1

Pitch: 13.100000000000001, Yaw: -134.20000000000002, Roll: 0.0

Pitch: 13.100000000000001, Yaw: -134.20000000000002, Roll: 0.0

Pitch: 13.100000000000001, Ya

ELEVATION GROUND GIMBAL TO DRONE

In [5]:
lat_gimbal = float(myHelper.mySeptentrioGPS.NMEA_buffer[-1]['Latitude'][0:2]) + float(myHelper.mySeptentrioGPS.NMEA_buffer[-1]['Latitude'][2:])/60
lon_gimbal = float(myHelper.mySeptentrioGPS.NMEA_buffer[-1]['Longitude'][0:3]) + float(myHelper.mySeptentrioGPS.NMEA_buffer[-1]['Longitude'][3:])/60
height_gimbal = float(myHelper.mySeptentrioGPS.NMEA_buffer[-1]['Antenna Alt above sea level (mean)'])

print('This is are the coordinates for the GROUND station\n')
print(f'LAT:{lat_gimbal}, LON: {lon_gimbal}, HEIGHT: {height_gimbal}')

# Point GROUND gimbal towards DRONE
lat_drone = 60.15893
lon_drone = 24.91091
height_drone = 40

lat_drone_planar, lon_drone_planar = myHelper.convert_DDMMS_to_planar(lon_drone, lat_drone, offset=None, epsg_in=4326, epsg_out=3901)
lat_gimbal_planar, lon_gimbal_planar = myHelper.convert_DDMMS_to_planar(lon_gimbal, lat_gimbal, offset=None, epsg_in=4326, epsg_out=3901)
        
position_drone = np.array([lon_drone_planar, lat_drone_planar, height_drone])
position_gimbal = np.array([lon_gimbal_planar, lat_gimbal_planar, height_gimbal])
print(f"[DEBUG]: POS_DRONE: {position_drone}, POS_GROUND: {position_gimbal}")
        
#d_mobile_drone_3D = np.linalg.norm(position_drone - position_gimbal)
d_mobile_drone_2D = np.linalg.norm(position_drone[:-1] - position_gimbal[:-1])
print(f"[DEBUG]: Azimuth distance from ground to drone station: {d_mobile_drone_2D}")
                
# Elevation angle of the drone referenced to GROUND station plane. 
# Is also the roll angle to send to the gimbal
roll_to_set = np.arctan2(height_drone - height_gimbal, d_mobile_drone_2D)
roll_to_set = int(np.rad2deg(roll_to_set)*10)

print(roll_to_set)

This is are the coordinates for the GROUND station

LAT:60.159963625, LON: 24.912774956666667, HEIGHT: 3.8317
[DEBUG]: POS_DRONE: [6.67374652e+06 3.38417057e+06 4.00000000e+01], POS_GROUND: [6.67385837e+06 3.38427774e+06 3.83170000e+00]
[DEBUG]: Azimuth distance from ground to drone station: 154.90484560038283
131


c:\Users\jvjulian\MyInstalledPrograms\envs\gimbalRS2\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\jvjulian\MyInstalledPrograms\envs\gimbalRS2\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\jvjulian\MyInstalledPrograms\envs\gimbalRS2\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<auth

POWER SPECTRUM MEASUREMENT CHECK

In [ ]:
data = []
n_samp = []
def iterate_until(iterable):
    
    """Returns an iterator that returns all keys or values
       of a (nested) iterable.
       
       Arguments:
           - iterable: <list> or <dictionary>
           - returned: <string> "key" or "value"
           
       Returns:
           - <iterator>
    """
    if isinstance(iterable, dict):
        for key, value in iterable.items():
            
            if not (isinstance(value, dict) or isinstance(value, list)):
                if key == '@id':
                    try:
                        n_samp.append(int(value))
                        data.append(iterable['@value'])
                    except:
                        1
                if key == '@TimeStamp':
                    1
            else:
                iterate_until(value)

    elif isinstance(iterable, list):
        #print(iterable)
        #data.append(iterable)
        for i in iterable:
            iterate_until(i)
      
    return data, n_samp

In [ ]:
from cgi import test
import xmltodict
import json
import pprint
import os
import matplotlib.pyplot as plt
import numpy as np

root_directory = "C://Users//jvjulian//OneDrive - Teknologian Tutkimuskeskus VTT//Documents//Measurement 1//"
test_folders = "20221012"

# Each folder has 498 files (except first file of first folder)
# Each file has around 650 points
# These are consecutive points

max_num_points_per_plot = 50000

for i in os.listdir(root_directory):
    if test_folders in i:
        this_folder = root_directory + i
        for j in os.listdir(this_folder):
            filename = this_folder + '//' + j

            with open(filename) as fd:
                try:
                    doc = xmltodict.parse(fd.read(), process_namespaces=True)
                except:
                    print('EXCEPTION on file ', filename)
            data, n_samp = iterate_until(doc)

y = [np.nan if i=='nan' else float(i) for i in data]

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(y)

In [ ]:
az_now = 1400
reset_ang_buffer = []
Naz = 12
for i in range(Naz):
    ang = int((i+1)*3600/Naz)
    ang = az_now + ang
    if ang > 1800:
        ang = ang - 3600
    if ang < -1800:
        ang = ang + 3600
    reset_ang_buffer.append(ang)
    
print(reset_ang_buffer)

reset_buffer = reset_ang_buffer[-2::-1]
reset_buffer.append(reset_ang_buffer[-1])

print(reset_buffer)           